In [6]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# kaggle dataset download
import kaggle

dataset = "shubhammaindola/harry-potter-books"

kaggle.api.dataset_download_files(dataset, path="harry_potter_books", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books


In [21]:
import re
import os

def clean_text(filename):

    file_path = os.path.join("harry_potter_books", filename)

    with open(file_path, 'r', encoding='utf-8') as file:
        book_text = file.read()

    cleaned_text = re.sub(r'\n', ' ', book_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    print("cleaned_" + filename, len(cleaned_text), "characters")
    
    with open("cleaned_" + filename, 'w', encoding="utf-8") as file:
        file.write(cleaned_text)

filename_list = ["02 Harry Potter and the Chamber of Secrets.txt"]

for filename in filename_list:
    clean_text(filename)

cleaned_02 Harry Potter and the Chamber of Secrets.txt 488771 characters


In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

text = "Harry Potter is a wizard."

token_en = tokenizer.encode(text)
token_de = tokenizer.decode(token_en)
tokens = tokenizer.convert_ids_to_tokens(token_en)

print(token_en)
print(token_de)

for token_e, token in zip(token_en, tokens):
    print(f"{token_e} -> {token}")

c:\Users\rladn\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[101, 4302, 10693, 2003, 1037, 10276, 1012, 102]
[CLS] harry potter is a wizard. [SEP]
101 -> [CLS]
4302 -> harry
10693 -> potter
2003 -> is
1037 -> a
10276 -> wizard
1012 -> .
102 -> [SEP]


In [9]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install dataloader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

class Toeknizer_to_harrypotter(Dataset):
    def __init__(self, txt, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

with open("cleaned_02 Harry Potter and the Chamber of Secrets.txt", 'r', encoding="utf-8") as file:
    txt = file.read()

dataset = Toeknizer_to_harrypotter(txt, max_length=32, stride=4)

train_dataloder = DataLoader(dataset, batch_size=128, shuffle=True, drop_last=True)
    



Token indices sequence length is longer than the specified maximum sequence length for this model (122530 > 512). Running this sequence through the model will result in indexing errors


In [14]:
dataiter = iter(train_dataloder)

x, y = next(dataiter)

print(tokenizer.decode(x[0].tolist()))
print(tokenizer.decode(y[0].tolist()))

fiercely. “ one of my best friends is muggle - born ; she ’ ll be first in line if the chamber really has been opened — ” “ harry
. “ one of my best friends is muggle - born ; she ’ ll be first in line if the chamber really has been opened — ” “ harry potter


In [15]:
VOCAB_SIZE = len(tokenizer)
CONTEXT_LENGTH = 128
EMB_DIM = 768
NUM_HEADS = 12
NUM_LAYERS = 12
DROP_RATE = 0.1
QKV_BIAS = False

In [18]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()

        assert d_out % NUM_HEADS == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.head_dim = d_out // NUM_HEADS

        self.W_query = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.W_key = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.W_value = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(DROP_RATE)
        self.register_buffer('mask', torch.triu(torch.ones(CONTEXT_LENGTH, CONTEXT_LENGTH), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        queries = queries.view(b, num_tokens, NUM_HEADS, self.head_dim)
        keys = keys.view(b, num_tokens, NUM_HEADS, self.head_dim)
        values = values.view(b, num_tokens, NUM_HEADS, self.head_dim)

        queries = queries.transpose(1, 2)
        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.reshape(b, num_tokens, self.d_out)

        return self.out_proj(context_vec)


class TransformerBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=EMB_DIM,
            d_out=EMB_DIM
        )

        self.ff = FeedForward()
        self.norm1 = nn.LayerNorm(EMB_DIM)
        self.norm2 = nn.LayerNorm(EMB_DIM)
        self.drop_shortcut = nn.Dropout(DROP_RATE)

    def forward(self,x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        return x
    

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(EMB_DIM, 4 * EMB_DIM),
            nn.ReLU(),
            nn.Linear(4 * EMB_DIM, EMB_DIM)
        )

    def forward(self, x):
        return self.layers(x)    


class TransformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.tok_emb = nn.Embedding(VOCAB_SIZE, EMB_DIM)
        self.pos_emb = nn.Embedding(CONTEXT_LENGTH, EMB_DIM)
        self.drop_emb = nn.Dropout(DROP_RATE)

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock() for _ in range(NUM_LAYERS)]
        )

        self.final_norm = nn.LayerNorm(EMB_DIM)
        self.out_head = nn.Linear(EMB_DIM, VOCAB_SIZE, bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

torch.manual_seed(123)
model = TransformerModel()
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

cpu


In [22]:
tokens_seen, global_step = 0, -1

losses = []

for epoch in range(10):
    model.train()

    epoch_loss = 0
    for input_batch, target_batch in train_dataloder:
        optimizer.zero_grad()
        input_batch, target_batch = input_batch.to(device), target_batch.to(device)

        logits = model(input_batch)
        loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        tokens_seen += input_batch.numel()
        global_step += 1

        if global_step % 1000 == 0:
            print(f"Tokens seen: {tokens_seen}")


    avg_loss = epoch_loss / len(train_dataloder)
    losses.append(avg_loss)
    print(f"Epoch: {epoch + 1}, loss: {avg_loss}")
    torch.save(model.state_dict(), "model_" + str(epoch + 1).zfill(3) + ".pth")

Tokens seen: 4096


KeyboardInterrupt: 